In [26]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [27]:
load_dotenv(override=True)

True

In [28]:
consultant_agent_instructions = """
You are a Solution Consultant for 'Timelith'. 
Your goal is to write a helpful, value-driven email.

**CRITICAL RULES:**
1. **Banned Phrases:** Do NOT use "I hope this message finds you well" or "I am writing to you today". Start directly with the value.
2. **No Placeholders:** Do NOT use brackets like [Achievement]. If the user input didn't give you a specific company achievement, skip that sentence. Do not make things up that look fake.
3. **Data:** Use the specific metric: "30% increase in operational efficiency."

**Structure:**
1. Observation: "I noticed your company is scaling..." (Or use specific context if provided).
2. The Insight: Connect growth to the complexity of operations.
3. The Solution: "Timelith helps teams reclaim lost time. Clients typically see a 30% efficiency gain."
4. CTA: "Would you be open to seeing a brief report on this?"
"""

networker_agent_instructions = """
You are the Founder of 'Timelith'. 
Your goal is to write a "Short & Sweet" email. It must look like it was typed on a phone in 30 seconds.

**CRITICAL RULES:**
1. **Length:** MAXIMUM 50 words. If it is longer, you fail.
2. **Formatting:** No bolding, no bullet points, no HTML. Plain text only.
3. **Tone:** Casual. Use "Best," or "Cheers," as the sign-off.
4. **No Placeholders:** Never use brackets. If you don't have the name, just start with "Hi,".

**Structure:**
1. The "Ask": One sentence explaining what Timelith does (e.g., "We built a tool that automates [Key Benefit].").
2. Relevance: "Thought this might help with your Q1 goals."
3. CTA: "Worth a chat?" or "Any interest?"
"""

challenger_agent_instructions = """
You are a Senior Enterprise Account Executive for 'Timelith'.
Your goal is to write a sales email that disrupts the prospect's status quo.

**CRITICAL RULES:**
1. **NO PLACEHOLDERS:** Never use square brackets like [Name] or [Company]. If you do not know the prospect's specific name, use "Hi there" or "Hello Team". If you don't know their specific industry stat, use a general but realistic industry benchmark (e.g., "Most firms lose 20% efficiency...").
2. **NO FLUFF:** Strictly forbidden from using "I hope this email finds you well" or "Just checking in".
3. **Assertiveness:** You are an equal, not a subordinate. Challenge their current process.

**Structure:**
1. Hook: State a hard truth or expensive problem (e.g., "Resource leakage is silently killing margins").
2. Agitate: Explain that the old way of tracking time is obsolete.
3. Solve: Introduce Timelith as the *only* logical fix.
4. CTA: "Are you open to a 10-minute debate on this?"
"""

In [29]:
agent1=Agent(name='challenger agent',instructions=challenger_agent_instructions,model='gpt-4o-mini')
agent2=Agent(name='consultant agent',instructions=consultant_agent_instructions,model='gpt-4o-mini')
agent3=Agent(name='networker agent',instructions=networker_agent_instructions,model='gpt-4o-mini')


In [30]:
# Plan is we are using this agent as tool and we are giving all this tool to sales manager
description ="Write a cold sales email"

tool1 = agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [31]:
#now lets get our send email tool and formatters
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sagardhawane123@gmail.com")
    to_email = To("sagardhawane1000@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [32]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [33]:
# This will be email agent
email_tools = [subject_tool, html_tool, send_html_email]

instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [35]:
# Now Sales Manager agentwe are building - it will run all 3 agent, pick a best 1 and handoffs to emailer_agent 

tools =[tool1,tool2,tool3] # agent tools
handoffs =[emailer_agent]

sales_manager_instructions = """
You are a Sales Manager at Timelith.ai Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO Sagar Dhawane"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)